In [2]:
# Getting required Libraries
import selenium
import os
import requests
import pandas as pd
import datetime
import re
import json
import certifi
import time
from time import sleep
from selenium import webdriver 
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import PyPDF2

In [17]:
# URL of the webpage containing the speech
url = 'https://www.president.go.ke/speeches/'

# Sending a request to the website with SSL verification disabled
response = requests.get(url, verify = False)

# Checking if the request was successful
if response.status_code == 200:
    # Parsing the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Finding the element containing the speech
    speech_element = soup.find('div', {'class': 'brxe-section'})

    # Checking if the speech element is found
    if speech_element:
        # Extracting the text from the speech element
        speech_text = speech_element.get_text()
        print(speech_text)
    else:
        print("Speech element not found on the webpage.")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


Speech element not found on the webpage.


C:\Users\Ernest\AppData\Roaming\Python\Python311\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.president.go.ke'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
import re

# Sample data
data = """
DURING THE OPENING OF THE 3RD SITTING OF THE 3RD SESSION OF THE 5TH EAST AFRICAN LEGISLATIVE ASSEMBLY
March 5, 2024

ON THE REPORT OF THE CAHOSCC MEETING OF 17TH FEBRUARY 2024 TABLED FOR ADOPTION AT THE AFRICAN UNION GENERAL ASSEMBLY
February 18, 2024

DURING AN EVENT CO-HOSTED BY THE GOVERNMENT OF KENYA AND THE INTERNATIONAL VACCINE INSTITUTE ON THE SIDELINES OF 37TH ORDINARY SESSION OF THE ASSEMBLY OF THE AFRICAN UNION IN ADDIS ABABA, ETHIOPIA.
February 18, 2024

AT THE MEETING OF THE COMMITTEE OF AFRICAN HEADS OF STATE AND GOVERNMENT ON CLIMATE CHANGE
February 17, 2024

Presidential Dialogue on the Global Financial Institutions Reform, Addis Ababa, 17th February, 2024 
February 17, 2024

AT THE HIGH-LEVEL EVENT ON CLIMATE FINANCE FOR AGRICULTURE AND FOOD SECURITY
February 16, 2024

Despatch from Cabinet – 14.02.2024
February 14, 2024

THE WORLD GOVERNMENTS SUMMIT IN DUBAI, UAE
February 13, 2024

FOLLOWING A BILATERAL MEETING WITH H. E. FUMIO KISHIDA, PRIME MINISTER OF JAPAN, ON STATE VISIT TO JAPAN
February 8, 2024

DURING THE KENYA-POLAND HIGH-LEVEL ECONOMIC DIALOGUE
February 6, 2024
"""

# Define a regular expression pattern to extract dates and events
pattern = re.compile(r'(.+?)\n([A-Za-z]+\s\d{1,2},\s\d{4})')

# Find all matches in the data
matches = pattern.findall(data)

# Display the extracted information
for match in matches:
    event = match[0].strip()
    date = match[1]
    print(f"Date: {date}, Event: {event}")


Date: March 5, 2024, Event: DURING THE OPENING OF THE 3RD SITTING OF THE 3RD SESSION OF THE 5TH EAST AFRICAN LEGISLATIVE ASSEMBLY
Date: February 18, 2024, Event: ON THE REPORT OF THE CAHOSCC MEETING OF 17TH FEBRUARY 2024 TABLED FOR ADOPTION AT THE AFRICAN UNION GENERAL ASSEMBLY
Date: February 18, 2024, Event: DURING AN EVENT CO-HOSTED BY THE GOVERNMENT OF KENYA AND THE INTERNATIONAL VACCINE INSTITUTE ON THE SIDELINES OF 37TH ORDINARY SESSION OF THE ASSEMBLY OF THE AFRICAN UNION IN ADDIS ABABA, ETHIOPIA.
Date: February 17, 2024, Event: AT THE MEETING OF THE COMMITTEE OF AFRICAN HEADS OF STATE AND GOVERNMENT ON CLIMATE CHANGE
Date: February 17, 2024, Event: Presidential Dialogue on the Global Financial Institutions Reform, Addis Ababa, 17th February, 2024
Date: February 16, 2024, Event: AT THE HIGH-LEVEL EVENT ON CLIMATE FINANCE FOR AGRICULTURE AND FOOD SECURITY
Date: February 14, 2024, Event: Despatch from Cabinet – 14.02.2024
Date: February 13, 2024, Event: THE WORLD GOVERNMENTS SUMMIT

In [19]:
url = "https://www.president.go.ke/speeches/"

# Disabling SSL verification
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.text, 'html.parser')

entries = soup.find_all("div", class_="brxe-kvryow brxe-block")

for entry in entries:
    title = entry.find("h2").text
    date = entry.find("span", class_="date").text
    content_url = entry.find("a")["href"]
    print(content_url)

    # Making request to the content_url to get the full speech content
    content_response = requests.get(content_url, verify=False)
    content_soup = BeautifulSoup(content_response.text, 'html.parser')

    # Extracting the speech content based on the HTML structure of the content page
    speech_content = content_soup.find("div", class_="brxe-kvryow brxe-block").text

    # Print or store the extracted data as needed
    print(f"Title: {title}\nDate: {date}\nSpeech Content: {speech_content}\n")

C:\Users\Ernest\AppData\Roaming\Python\Python311\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.president.go.ke'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [21]:
def download_pdf(url, folder):
    response = requests.get(url, verify=False)
    if response.status_code == 200:
        filename = os.path.join(folder, os.path.basename(url))
        with open(filename, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded: {filename}")
    else:
        print(f"Failed to download: {url}")

def scrape_pdf_links(url, folder):
    response = requests.get(url, verify=False)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        pdf_links = [urljoin(url, a['href']) for a in soup.find_all('a', href=True) if a['href'].lower().endswith('.pdf')]

        for pdf_link in pdf_links:
            download_pdf(pdf_link, folder)
    else:
        print(f"Failed to fetch webpage: {url}")

website_url = "https://www.president.go.ke/speeches/"
download_folder = 'pdf_downloads'

os.makedirs(download_folder, exist_ok=True)

scrape_pdf_links(website_url, download_folder)

C:\Users\Ernest\AppData\Roaming\Python\Python311\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.president.go.ke'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Ernest\AppData\Roaming\Python\Python311\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.president.go.ke'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Downloaded: pdf_downloads\DURING-THE-INTERNATIONAL-WOMENS-DAY-CELEBRATIONS.pdf


C:\Users\Ernest\AppData\Roaming\Python\Python311\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.president.go.ke'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Downloaded: pdf_downloads\DURING-THE-OPENING-OF-THE-3RD-SITTING-OF-THE-3RD-SESSION-OF-THE-5TH-EAST-AFRICAN-LEGISLATIVE-ASSEMBLY.pdf


C:\Users\Ernest\AppData\Roaming\Python\Python311\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.president.go.ke'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

Downloaded: pdf_downloads\Despatch-from-Cabinet-14.02.2024.pdf


C:\Users\Ernest\AppData\Roaming\Python\Python311\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.president.go.ke'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Downloaded: pdf_downloads\0224-WORLD-GOVERNMENT-SUMMIT-UPDATED-REVISED-FINAL-2-EN.pdf


C:\Users\Ernest\AppData\Roaming\Python\Python311\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.president.go.ke'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Downloaded: pdf_downloads\FOLLOWING-A-BILATERAL-MEETING-WITH-H.-E.-FUMIO-KISHIDA-PRIME-MINISTER-OF-JAPAN-ON-STATE-VISIT-TO-JAPAN.pdf


In [ ]:
def scrape_pdf_links_recursive(url, folder):
    page_number = 1
    while True:
        current_url = f"{url}page/{page_number}/" if page_number > 1 else url
        response = requests.get(current_url, verify=False)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extracting links with .pdf extension
            pdf_links = [urljoin(url, a['href']) for a in soup.find_all('a', href=True) if a['href'].lower().endswith('.pdf')]

            # Downloading each PDF file
            for pdf_link in pdf_links:
                download_pdf(pdf_link, folder)

            # Check for the next page link
            next_page_link = soup.find('a', class_='next page-numbers')
            if not next_page_link:
                break  # No more pages, exit the loop
            else:
                page_number += 1
        else:
            print(f"Failed to fetch webpage: {current_url}")
            break  # Exiting the loop on error



website_url = "https://www.president.go.ke/speeches/page/1/"
download_folder = 'pdf_downloads'

os.makedirs(download_folder, exist_ok=True)

scrape_pdf_links_recursive(website_url, download_folder)
